# **A. Pre-processing**

## Acquiring the TACO dataset
TACO is a growing image dataset of waste in the wild. It contains images of litter taken under diverse environments: woods, roads and beaches. These images are manually labeled and segmented according to a hierarchical taxonomy to train and evaluate object detection algorithms. Currently, images are hosted on
Flickr and we have a server that is collecting more images and annotations.

Get more information [here](https://github.com/pedropro/TACO).

In [ ]:
# 0. Clear Data
import os, shutil
if os.path.exists('sample_data/'): shutil.rmtree('sample_data/')

In [ ]:
# 1. Installing required libraries
!pip install pillow
!pip install requests
!pip install jupyter
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install graphviz
!pip install Cython

In [ ]:
# 2.1 Clone & Download dataset
!git clone https://github.com/pedropro/TACO
%cd /content/TACO
!python3 download.py

In [ ]:
# 2.2 OR Upload TACO.zip with 2-3 images (for simplicity's sake)
%cd /content
!unzip /content/TACO.zip

## Organize files & folders

In [ ]:
# 3. Move all images from multiple batch folders to a new single folder.
import os
import shutil

root_path = 'TACO/'
data_path = root_path + 'data/'
image_path = data_path + 'images/'

# Create new folder 'images'
%cd /content/
os.mkdir(image_path)

for batch in os.listdir(data_path):
    if os.path.isdir(data_path) and 'batch' in batch:
        for image in os.listdir(data_path + batch):
            shutil.move(data_path + batch + '/' + image, image_path + image)
        shutil.rmtree(data_path + batch)

## Load & Visualize annotations

The TACO dataset information is represented in a COCO format. To select and show dataset images with respective masks, we make use of the COCO API.

This sections uses the annotations.json file to generate a mask for every image in this dataset.

In [ ]:
# 4. Export Image Masks from JSON format.
from PIL import Image, ExifTags
from pycocotools.coco import COCO
from matplotlib.patches import Polygon, Rectangle
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
import numpy as np
import colorsys
import imageio
import shutil
import random
import pylab
import json
import os

# Paths
root_path = 'TACO/'
data_path = root_path + 'data/'
image_path = data_path + 'images/'
label_path = data_path + 'labels/'
anns_file_path = data_path + 'annotations.json'
pylab.rcParams['figure.figsize'] = (28,28)

# Read annotations
with open(anns_file_path, 'r') as f:
    dataset = json.loads(f.read())
categories = dataset['categories']
anns = dataset['annotations']
imgs = dataset['images']

# Clearing garbage (pun intented)
if (os.path.exists(label_path)): 
    shutil.rmtree(label_path)

# Create mask directory
os.mkdir(label_path)

# Generate mask for every image in this dataset
for image_filepath in os.listdir(image_path):
    # Obtain Exif orientation tag code
    for orientation in ExifTags.TAGS.keys():
        if ExifTags.TAGS[orientation] == 'Orientation':
            break

    # Loads dataset as a coco object
    coco = COCO(anns_file_path)

    # Find image id
    img_id = -1
    for img in imgs:
        check_image = img['file_name'].rpartition('/')[2]
        if check_image == image_filepath:
            img_id = img['id']
            break

    # Show image and corresponding annotations
    if img_id == -1:
        print('Incorrect file name')
    else:
        # Create binary mask from annotations.json and export as image
        anns_img = np.zeros((img['height'],img['width']))
        for ann in anns:
            if ann['image_id'] == img_id:
                a = coco.annToMask(ann)*ann['category_id']
                anns_img = np.maximum(anns_img,a)

        imageio.imwrite(label_path + image_filepath, anns_img)


# **B. Training**

### 1. Setting up training process

In [ ]:
import os, shutil

%cd Mask_RCNN/

model_path = '/content/Mask_RCNN'
if os.path.exists(model_path): shutil.rmtree(model_path)

!git clone https://github.com/matterport/Mask_RCNN /content/Mask_RCNN
!python3 setup.py install

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("/content/Mask_RCNN")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

In [ ]:
# Configurations

class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

In [ ]:
# Create Model and Load Trained Weights

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [ ]:
# COCO Class names

# Index of the class in the list is its ID.
class_names = ['BG', 'garbage']

In [ ]:
# Run Object Detection

# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]
image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

# Run detection
results = model.detect([image], verbose=1)

# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])

### 2. Train


# **C. Testing**